In [32]:
import time
start_time = time.time()


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import collections
import math
from numpy import linalg as LA
import re
from textblob import TextBlob
#Reading all the reviews and merging it
df1 = pd.read_excel('/Users/ksharath/Downloads/aws.xlsx',engine='openpyxl')
df2 = pd.read_excel('/Users/ksharath/Downloads/gcloud_text.xlsx',engine='openpyxl')
df3 = pd.read_csv('/Users/ksharath/Downloads/azure_cleaned_reviews.csv')
df3.columns = df1.columns
df = pd.concat([df1, df2, df3])

df['text'] = df [0]

In [33]:
#df.head()

***Pre processing for reviews text***

In [34]:

#preprocess
def pre_process(x):
    stop_words = set(stopwords.words('english'))
    #stop_words= set(stop_words.update(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]))
    #unwanted = ['would','should','']
    #word_tokens = word_tokenize(example_sent)
    filtered_sentence = [w for w in x if not w.lower() in stop_words]
    return filtered_sentence

#Tokenize
df[0] = df[0].apply(lambda x: re.sub(r"[^a-zA-Z0-9]", " ", x.lower()).split())
df[0] = df[0].apply(lambda x: pre_process(x))
                    
df.head()

,0,Unnamed: 0,text
0,"[aws, love, lambda]",0.0,AWS we love Lambda
1,"[multiple, locations, databases, services, ava...",1.0,Multiple locations and databases . Some servic...
2,"[aws, great, cms, iam, easy, use, add, users, ...",3.0,AWS has a great CMS IAM were easy to use to ad...
3,"[2, factor, authentication, sometime, would, r...",5.0,2 factor authentication sometime would rest mu...
4,"[developer, certifications]",6.0,Had more Developer with certifications


In [35]:
#Prepare a word set
new_list = df[0].tolist()
wordset=[]
for i in new_list:
    wordset = np.union1d(wordset,i)

***Function for TF***

In [36]:
def calculateTF(wordset,bow):
  termfreq_diz = dict.fromkeys(wordset,0)
  counter1 =  dict(collections.Counter(bow))
  for w in bow:
    termfreq_diz[w]=counter1[w]/len(bow)
  return termfreq_diz

In [37]:
#Create a dictionary of Topic frequenxy TF
doc_dic ={}
for doc in df[0].to_list():
    doc_dic[tuple(doc)] = calculateTF(wordset,doc)
    
    

***Function for IDF***

In [38]:
def calculate_IDF(wordset,bow):
    d_bow = {'bow_{}'.format(i):list(set(b)) for i,b in enumerate(bow)}
    N=len(d_bow.keys())
    l_bow = []
    for b in d_bow.values():
      l_bow+=b
    counter = dict(collections.Counter(l_bow))
    idf_diz=dict.fromkeys(wordset,0)
    for w in wordset:
      idf_diz[w] = np.log((1+N)/(1+counter[w]))+1
    return idf_diz

In [39]:
idf_diz = calculate_IDF(wordset,df[0].tolist())

df_idf = pd.DataFrame([idf_diz])
#df_idf

In [40]:
def calculate_TF_IDF(wordset,tf_diz,idf_diz):
    tf_idf_diz = dict.fromkeys(wordset,0)
    for w in wordset:
       tf_idf_diz[w]=tf_diz[w]*idf_diz[w]
    tdidf_values = list(tf_idf_diz.values())
    l2_norm = LA.norm(tdidf_values)   
    tf_idf_norm = {w:tf_idf_diz[w]/l2_norm for w in wordset}
    return tf_idf_norm

In [41]:
tfidf_doc_dic= {}
for doc in df[0].to_list():
    
    tfidf_doc_dic[tuple(doc)] = calculate_TF_IDF(wordset,doc_dic[tuple(doc)],idf_diz)

**Let's also create Sentiment scoring here**

In [42]:
df['calculated'] = df['text'].apply(lambda x: (TextBlob(x).sentiment.polarity))
df.head()

,0,Unnamed: 0,text,calculated
0,"[aws, love, lambda]",0.0,AWS we love Lambda,0.500000
1,"[multiple, locations, databases, services, ava...",1.0,Multiple locations and databases . Some servic...,0.384091
2,"[aws, great, cms, iam, easy, use, add, users, ...",3.0,AWS has a great CMS IAM were easy to use to ad...,0.555556
3,"[2, factor, authentication, sometime, would, r...",5.0,2 factor authentication sometime would rest mu...,0.000000
4,"[developer, certifications]",6.0,Had more Developer with certifications,0.500000


**Tagging the topics from TF-IDF with a final score**

In [43]:
json_l =[]
for doc in df[0].to_list():
    json_l.append(tfidf_doc_dic[tuple(doc)])
final_df = pd.DataFrame(json_l)

final_df_stat = final_df.mean()
output = pd.DataFrame(final_df_stat)
#Here we can get top 50 or top 100 topics

#output = pd.DataFrame(final_df_stat).nlargest(50, 0)
final_list =[]
for i in output.iterrows():
    #print("Iterating for topic: %s"%i[0])
    sent_score = 0
    for j in df.iterrows():
        if i[0] in j[1]['text'].lower():
            sent_score = sent_score + j[1]['calculated']
    #print("Original TF-IDF score")
    #print(i[1][0])
    #print("TF-IDF with sentiment score")
    #print(i[1][0]*sent_score)
    #print("\n")
    final_list.append([ i[0], i[1][0], i[1][0]*sent_score])
    

In [44]:
model_output = pd.DataFrame(final_list)
model_output.columns = ["Topic_name", "TFIDF", "TFIDF_with_sentiment"]
#Let's write the output to the output file now
model_output.to_csv("~/Desktop/output_TFIDF.csv")
print("--- %s seconds ---" % (time.time() - start_time))

--- 196.0186710357666 seconds ---


**Sentiment Analyzer**

In [384]:
#!pip install spacy